In [1]:
import polib
from openai import OpenAI
import time
from dotenv import load_dotenv
import os
from tqdm import tqdm
import glob

In [2]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [6]:
load_dotenv(override=True)

def get_completion(system_message, user_message, model="gpt-3.5-turbo"):
    client = OpenAI()
    
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
    completion = client.chat.completions.create(
        model=model,
        messages=messages
    )
    return completion.choices[0].message.content

def translate_text_to_japanese(text):
    system_message = "あなたはデジタルデータの長期保存に関する専門家です。Archivematicaに関するドキュメントを翻訳しています。以下のテキストを日本語に翻訳してください。"

    response = get_completion(system_message, text, model="gpt-4") # , model="gpt-4"
    return response

def translate_po_file(input_po_path, output_po_path):
    """`.po`ファイルを読み込み、各メッセージを日本語に翻訳して新しい`.po`ファイルに保存します。"""
    po = polib.pofile(input_po_path)
    for entry in tqdm(po):
        if not entry.msgstr:
            translated_text = translate_text_to_japanese(entry.msgid)
            entry.msgstr = translated_text
            # APIリクエストごとに適度な時間を置いてレートリミットに配慮
            time.sleep(1)
            po.save(output_po_path)

files = glob.glob("/Users/nakamura/git/kim/archivematica-storage-service-docs/locales/en/LC_MESSAGES/**/*.po", recursive=True)

files.sort()

for i in range(len(files)):
    input_po_path = files[i]

    print(input_po_path, i + 1, "of", len(files))

    '''
    if "fixity.po" in input_po_path or "administrators.po" in input_po_path:
        continue
    '''

    if "administrators.po" not in input_po_path:
        continue

    # 入力と出力の`.po`ファイルパスを指定
    # input_po_path = '/Users/nakamura/git/kim/archivematica-storage-service-docs/locales/en/LC_MESSAGES/administrators.po'
    # output_po_path = '/Users/nakamura/git/kim/archivematica-storage-service-docs/locales/ja/LC_MESSAGES/administrators.po'

    output_po_path = input_po_path.replace('/en/', '/ja/')

    # `.po`ファイルの翻訳を実行
    translate_po_file(input_po_path, output_po_path)

ValueError: too many values to unpack (expected 2)